In [1]:
import sys
sys.path.append("..")


In [2]:
from data.dataloader import LowLightDataModule
from model.blocks.contourlet import ContourletTransform
from model.blocks.directionalfusion import DirectionalFusion
from utils.utils import show_batch


In [3]:
data_module = LowLightDataModule(
    train_dir="../data/1_train",
    valid_dir="../data/2_valid",
    bench_dir="../data/3_bench",
    infer_dir="../data/4_infer",
    image_size=640,
    batch_size=4,
    num_workers=4
)

data_module.setup()


In [4]:
train_dataloader = data_module.train_dataloader()


In [5]:
train_low, train_high = next(iter(train_dataloader))
print(train_low.shape)
print(train_high.shape)
# show_batch(images=train_low)
# show_batch(images=train_high)


torch.Size([4, 3, 640, 960])
torch.Size([4, 3, 640, 960])


In [6]:
contourlet = ContourletTransform(
    in_channels=3,
    num_levels=4,
    filter_size=5,
    sigma=1.0,
    omega_x=0.25,
    omega_y=0.25,
)

directioanlfusion = DirectionalFusion(
    in_channels=3,
    hidden_channels=64,
    num_levels=4,
    squeeze_ratio=0.3
)


c:\Users\jungih\dev\ContourDiffusion\.venv\lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [7]:
train_low = train_low.cuda()
contourlet = contourlet.cuda()
directioanlfusion = directioanlfusion.cuda()

pyramid, subbands = contourlet(train_low)
fusion = directioanlfusion(subbands)


In [8]:
for i in fusion:
    print(i.shape, i.device)


torch.Size([4, 512, 80, 120]) cuda:0
torch.Size([4, 256, 160, 240]) cuda:0
torch.Size([4, 128, 320, 480]) cuda:0
torch.Size([4, 64, 640, 960]) cuda:0
